# Домашнее задание 2 по ML
Нужно написать python класс градиентного бустинга и побить другую модель на предоставленном baseline

## Критерии оценки
- Ваш ноутбук будет запущен через ```run all``` - он не должен упасть (допускается падение из-за отсутствия библиотеки, которую можно поставить через pip install)
- Вот этот код (внизу ноутбука) ```assert imp_my_little_model > imp_baseline_model``` не вызвал ошибок (успешно отработал)

- реализованы следующие гиперпараметры
  - вы реализовали гиперпараметр ```learning_rate```
  - вы реализовали гиперпараметр ```n_estimators```
  - вы реализовали гиперпараметр ```max_depth```
  - вы реализовали гиперпараметр ```bagging_fraction```

- Вы реализовали [Huber loss function](https://ru.wikipedia.org/wiki/%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F_%D0%BF%D0%BE%D1%82%D0%B5%D1%80%D1%8C_%D0%A5%D1%8C%D1%8E%D0%B1%D0%B5%D1%80%D0%B0) - она записана как отдельная def функция вне класса - и используется в вашем классе для расчета

----
*Для успешной сдачи дз нужно выполнить полностью каждый пункт выше*

- оценка 5 будет поставлена, если каждый пункт выполнен без недочетов
- оценка 4 будет поставлена, если будет найден один недочет
- незачет будет пославлен, если недочетов будет два или более
- незачет будет пославлен, если какой либо пункт не выполнен


In [ ]:
%autosave 60

Autosaving every 60 seconds


In [ ]:
# pip install shap

import numpy as np
import shap

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 2.3 MB/s eta 0:00:00


In [ ]:
np.random.seed(42)

In [ ]:
data, target = shap.datasets.california()
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

### Не меняйте название для предскзаний preds_my_little_model, иначе не получится сдать это ДЗ (сломается код)

Некоторые правила
- Нельзя использовать никакие другие алгоритмы моделей внутри вашего класса, кроме DecisionTreeRegressor.
- Код вашего бустинга должен быть написан в классе, у класса должно быть два ожидаемых метода : ```fit``` и ```predict```.
- Нельзя менять датасет (и модифицировать тоже, например заполнять nan или применять scaler) или baseline модель
- Нельзя поднимать число n_estimators вашей модели выше 100 (чтобы результат был сравним с моделью-конкурентом ```GradientBoostingRegressor```)

# *это место для вашего кода* ↓↓↓


In [23]:
def huber_loss(y_true, y_pred, delta=1):
  '''Функция для вычисления ГРАДИЕНТА функции ошибки huber loss (так как в оптимизации нужно считать именно градиент ошибки)'''
  err = y_true - y_pred
  return np.where(abs(err) <= delta, err, delta * np.sign(err))

In [24]:
class MyGradBoosting:
    """Класс, описывающий градиентный бустинг"""
    def __init__(
        self,
        learning_rate=0.2,
        n_estimators=100,
        max_depth=5,
        random_state=0,
        bagging_fraction=0.9,
    ):
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.bagging_fraction = bagging_fraction


    def fit(self, X, y):
        n = len(X)
        self.trees = []   # Список обученных слабых моделей
        f = np.zeros(n)  # Начальный прогноз модели

        for i in range(self.n_estimators):
            loss = huber_loss(y, f)  # Градиент функции ошибки
            bagging_ind = np.random.choice(n, int(n * self.bagging_fraction), replace=True)  # Бэггинг записей

            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=self.random_state)
            tree.fit(X.iloc[bagging_ind], loss[bagging_ind])

            f +=  self.learning_rate * tree.predict(X)  # Пересчет прогноза модели
            self.trees.append(tree)


    def predict(self, samples):
        n = len(samples)
        predict = np.zeros(n) # Начальный прогноз

        for i in range(self.n_estimators):
            predict +=  self.learning_rate * self.trees[i].predict(samples)  # Пересчет прогноза

        return predict

In [25]:
my_little_model = MyGradBoosting()
my_little_model.fit(X_train, y_train)

preds_my_little_model = my_little_model.predict(X_test)

In [26]:
# самопроверки
assert preds_my_little_model.shape == y_test.shape, 'что-то не так с выходным размером предикта'

# *это место для вашего кода* ↑↑↑

### *ниже ничего менять не нужно*
## Это класс судья - он решит, какая модель оказалась лучше, ваша, или GradientBoostingRegressor из sklearn
Если ячейка ниже завершилась ошибкой, нужно поменять код вашей модели и попробовать еще раз, до тех пор, пока не получите сообщение "Ура, получилось!"

In [27]:
baseline_model = GradientBoostingRegressor(random_state=4, verbose=0)
baseline_model = baseline_model.fit(X_train, y_train)
preds_baseline_model = baseline_model.predict(X_test)
print('mape - ваша модель', mean_absolute_percentage_error(y_test, preds_my_little_model))
print('mape - baseline', mean_absolute_percentage_error(y_test, preds_baseline_model))

final_estimator = RandomForestRegressor(random_state=16)
final_estimator = final_estimator.fit(
    np.hstack((preds_baseline_model.reshape(-1, 1), preds_my_little_model.reshape(-1, 1))),
    y_test
)

imp_baseline_model, imp_my_little_model = final_estimator.feature_importances_
result_message = f"baseline важность: {imp_baseline_model:0.3f}; важность вашей модели: {imp_my_little_model:0.3f}"

assert imp_my_little_model > imp_baseline_model,  f'попробуй еще раз: {result_message}'
print('Ура, получилось!',  result_message)

mape - ваша модель 0.18522226037284975
mape - baseline 0.2152446498010688
Ура, получилось! baseline важность: 0.091; важность вашей модели: 0.909
